In [216]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [217]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [218]:
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


In [219]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), keep_dims=True)
    
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)),keep_dims=True)
    
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    
    
    return loss

In [220]:
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.01, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(loss.eval()))


loss = 137.733


In [221]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [285]:
database = {}
database["Rohan"] = img_to_encoding("images/Rohan.png", FRmodel)
#database["Rohan"] = img_to_encoding("images/TES.png",FRmodel)
database["Rohan"] =  img_to_encoding("images/rohandb.png",FRmodel)
database["Rohan"] = img_to_encoding("images/rohandb2.png",FRmodel)
database["Rohan"] = img_to_encoding("images/rtest3.png",FRmodel)
database["Rohan"] = img_to_encoding("images/rohandb3.png",FRmodel)
database["Rohan"] = img_to_encoding("images/rohandb4.png",FRmodel)
database["Rohan"] = img_to_encoding("images/rohandb5.png",FRmodel)
database["Rohan"] = img_to_encoding("images/rohandb6.png",FRmodel)
database["Rohan"] = img_to_encoding("images/rohandb7.png",FRmodel)
database["Rohan"] = img_to_encoding("images/te1.png",FRmodel)
database["Rohan"] = img_to_encoding("images/rohantest.png",FRmodel)
database["Neena"] = img_to_encoding("images/Neena.png",FRmodel)
database["Neena"] = img_to_encoding("images/neenadb.png",FRmodel)
database["Neena"] = img_to_encoding("images/ntest.png",FRmodel)
database["Neena"] = img_to_encoding("images/neenadb2.png",FRmodel)
database["Neena"] = img_to_encoding("images/neenadb3.png",FRmodel)
database["Neena"] = img_to_encoding("images/neenadb4.png",FRmodel)
database["Neena"] = img_to_encoding("images/neenadb11.png",FRmodel)
database["Neena"] = img_to_encoding("images/neenadb12.png",FRmodel)
database["Neena"] = img_to_encoding("images/test2.png",FRmodel)
database["Neena"] = img_to_encoding("images/neenadb10.png",FRmodel)
database["Bahaar"] = img_to_encoding("images/Bahaar.png",FRmodel)
database["Bahaar"] = img_to_encoding("images/bahaardb.png",FRmodel)
database["Bahaar"] = img_to_encoding("images/bahaardb1.png",FRmodel)
database["Bahaar"] = img_to_encoding("images/bahaardb2.png",FRmodel)
database["Bahaar"] = img_to_encoding("images/bahaardb3.png",FRmodel)
database["Bahaar"] = img_to_encoding("images/bahaardb4.png",FRmodel)
database["Bahaar"] = img_to_encoding("images/testdb.png",FRmodel)
database["Bahaar"] = img_to_encoding("images/btest2.png",FRmodel)
#database["Bahaar"]




In [286]:
database

{'Rohan': array([[  1.41259432e-01,   5.50870262e-02,   4.77159396e-02,
           6.85620727e-03,   5.65141328e-02,   1.61163598e-01,
           1.30260810e-01,  -7.53150955e-02,  -3.08801215e-02,
          -1.43794253e-01,  -1.41841657e-02,   4.76827398e-02,
           1.32140607e-01,   1.47174997e-02,   8.30279547e-04,
          -4.40417901e-02,   3.03265196e-03,   1.20907234e-05,
          -1.92416534e-01,   1.04323991e-01,   1.11037411e-01,
           4.09412570e-02,  -4.46907617e-02,   1.42291412e-01,
          -8.20010304e-02,  -1.02422267e-01,  -1.90330982e-01,
          -8.55620205e-02,  -9.44991410e-03,   1.60918385e-01,
          -4.05330360e-02,   1.36539653e-01,  -9.23220366e-02,
           1.42616481e-01,   2.90209111e-02,  -1.29260588e-04,
           5.98476864e-02,   1.01857983e-01,  -9.47255269e-03,
          -3.24795693e-02,   3.84563804e-02,  -2.45162230e-02,
           3.23590748e-02,  -1.29972056e-01,  -2.59295478e-02,
           1.44280186e-02,   5.27789816e-02,  

In [287]:
def verify(image_path, identity, database, model):
    
    
    encoding = img_to_encoding(image_path, model)
    

    dist = np.linalg.norm(encoding - database[identity])
    

    if dist < 0.7:
        print("It's " + str(identity) + ", welcome home!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False
        

        
    return dist, door_open

In [288]:
def who_is_it(image_path, database, model):
    encoding = img_to_encoding(image_path, model)
    
    
    min_dist = 100
    
    
    for (name, db_enc) in database.items():
        
        
        dist = np.linalg.norm(encoding - db_enc)

        #
        if dist < min_dist:
            min_dist = dist
            identity = name

    #
    
    if min_dist > 0.65:
        print("Not in the database./No face found")
    else:
        print (str(identity))
        return identity
       # print ("it's " + str(identity) + ", the distance is " + str(min_dist))
       # return min_dist, identity

In [331]:
who_is_it("images/neenadb11.png", database, FRmodel)

Neena


'Neena'

In [290]:
who_is_it("images/rohandb2.png", database, FRmodel)

Rohan


'Rohan'

In [291]:
who_is_it("images/tester.png", database, FRmodel)

Rohan


'Rohan'

In [292]:
who_is_it("images/rohantest.png", database, FRmodel)

Rohan


'Rohan'

In [293]:
who_is_it("images/bottle.png", database, FRmodel)

Not in the database./No face found


In [294]:
who_is_it("images/test2.png", database, FRmodel)

Rohan


'Rohan'

In [295]:
who_is_it("images/testdb.png", database, FRmodel)

Not in the database./No face found


In [296]:
who_is_it("images/kbh.png", database, FRmodel)

Rohan


'Rohan'

In [297]:
who_is_it("images/Bobby.png", database, FRmodel)

Not in the database./No face found


In [299]:
who_is_it("images/btest2.png", database, FRmodel)

Bahaar


'Bahaar'

In [300]:
who_is_it("images/rtest2.png", database, FRmodel)

Rohan


'Rohan'

In [301]:
who_is_it("images/tester.png", database, FRmodel)

Rohan


'Rohan'

In [77]:
from random import randint

In [215]:
from PIL import Image

img = Image.open(r'C:\Users\dell15\Desktop\FaceRec\Extracted\4268.jpg') # image extension *.png,*.jpg
new_width  = 96
new_height = 96
img = img.resize((new_width, new_height), Image.ANTIALIAS)
img.save(r'C:\Users\dell15\Desktop\FaceRec\Extracted\neenadb12.png')

In [98]:
CASCADE="haarcascade_frontalface_default.xml"
FACE_CASCADE=cv2.CascadeClassifier(CASCADE)

In [349]:
def trainer(filename):
    
    
    
    extracted = who_is_it(filename, database, FRmodel)
    
    if(extracted=='Neena'):
        
        
        image=cv2.imread(filename)
        image_grey=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        #faces = FACE_CASCADE.detectMultiScale(image_grey,scaleFactor=1.16,minNeighbors=5,minSize=(25,25),flags=0)
        #for x,y,w,h in faces:
        sub_img=image
        
        os.chdir(r"C:\Users\dell15\FolderNeena")
        cv2.imwrite(str(randint(0,10000))+".png",sub_img)
    
    if(extracted=='Rohan'):
        
        image=cv2.imread(filename)
        #image_grey=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        #faces = FACE_CASCADE.detectMultiScale(image_grey,scaleFactor=1.16,minNeighbors=5,minSize=(25,25),flags=0)
        #for x,y,w,h in faces:
        sub_img=image
        
        os.chdir(r"C:\Users\dell15\FolderRohan")
        cv2.imwrite(str(randint(0,10000))+".png",sub_img)
        #cv2.rectangle(image,(x,y),(x+w,y+h),(255, 255,0),2)
          #  img.save(r'C:\Users\dell15\adversarial\neena.png')
    
     
    if(extracted=='Bahaar'):
        
        image=cv2.imread(filename)
        #image_grey=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        #faces = FACE_CASCADE.detectMultiScale(image_grey,scaleFactor=1.16,minNeighbors=5,minSize=(25,25),flags=0)
        #for x,y,w,h in faces:
        sub_img=image
        
        os.chdir(r"C:\Users\dell15\FolderBahaar")
        cv2.imwrite(str(randint(0,10000))+".png",sub_img)
        #cv2.rectangle(image,(x,y),(x+w,y+h),(255, 255,0),2)
          #  img.save(r'C:\Users\dell15\adversarial\neena.png')
        
           
        



In [350]:
import glob

In [351]:
files = glob.glob(r'C:\Users\dell15\Desktop\FaceRec\Extracted\*.png')

In [352]:
for ele in files:
    trainer(ele)

Neena
Not in the database./No face found
Neena
Rohan
Neena
Rohan
Bahaar
Rohan
Rohan
Rohan
Bahaar
Not in the database./No face found


In [197]:
pwd

'C:\\Users\\dell15\\FolderBahaar'

In [198]:
os.chdir(r'C:\\Users\\dell15')

In [199]:
pwd

'C:\\Users\\dell15'

In [212]:
pwd

'C:\\Users\\dell15'

In [330]:
os.chdir(r'C:\\Users\\dell15')

In [317]:
pwd

'C:\\Users\\dell15'